# Film production.

# 1.0 Introduction

## 1.1 Business Understanding

One of the most well-known genres of art, film production has millions of spectators worldwide. There are many ways that movies can be communicated to viewers as a result of the development of various films genres over time.

Are budgets and box office grosses connected?


After observing that all the big companies were producing original video content, Microsoft made the decision to capitalize on the rising popularity of films. It is my duty to research the categories of movies that are doing the best at the box office and to translate findings into useful advice. I intend to conduct studies and then use the results to develop data-driven conclusions that will help my client.

## 1.2 Data Understanding

The dataset for this project was compiled from several sources. There are five movie datasets from:
* Box Office Mojo data
* IMDB  data  
* popular movies data
* top rated movies data

For this project, perfomance of a movie will be based on the return on investment (ROI). The formulae used to calculate ROI is: (collection from box office)-(the budget cost). This will be used to classify if the movie perfomed well or not.

The data sets had other columns which were dropped and the analysis will we concentarded on mainly,rating,title,genre,runtime and genre. The first rows to be dropped were those that had any missin value in the revenue column since they could not be classified based on ROI.

definitions of terms used during the eda process

# 2.0 Data Preparation Process

## 2.1 Loading Libraries

In [ ]:
#pip install WordCloud

In [ ]:
# Importing libraries
import pandas as pd
import numpy as np

import sqlite3

from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline
from wordcloud import WordCloud 

# Read sqlite query results into a pandas DataFrame
conn = sqlite3.Connection("Data/im.db")

## 2.2 Loading datasets

In [ ]:
# Loading of the data sets.
  
#box office gross dataset
box_office_df = pd.read_csv('Data/bom.movie_gross.csv.gz')

#Teddy Newton movie budgets dataset
movie_budgets_df = pd.read_csv('Data/tn.movie_budgets.csv.gz')

#the movie data base dataset
tmdb_df = pd.read_csv('Data/tmdb.movies.csv.gz')

#Rotten Tomatoes reviews dataset
rt_reviews_df = pd.read_csv('Data/rt.reviews.tsv.gz', sep='\t', encoding = 'unicode_escape')

#Rotten Tomatoes movie info dataset
rt_movie_info_df = pd.read_csv('Data/rt.movie_info.tsv.gz', sep='\t', encoding = 'unicode_escape')


In [ ]:
 #Read sqlite query results into a pandas DataFrame
#joining the two tables using the movie_id key

joint_query = """
SELECT *
FROM movie_basics
JOIN movie_ratings USING (movie_id)
;
"""
joint_query = pd.read_sql(joint_query, conn)
joint_query.tail(3)

# 3.0 Data Cleaning

### 3.0.1 combining of the data sets to a final data set

In [ ]:
#the data sets were all combied on common keys before cleaning of the data

joint_query.dropna(inplace = True)

box_office_df.dropna(inplace = True)

box_office_df.rename(columns = {'title':'original_title'}, inplace = True)

combined_data_1 = pd.merge(joint_query, box_office_df, how='inner', on='original_title')

movie_budgets_df.dropna(inplace = True)

movie_budgets_df.rename(columns = {'movie':'original_title'}, inplace = True)
 
# Find the columns that aren't in the first DataFrame
different_cols = combined_data_1.columns.difference(movie_budgets_df.columns)
 
# Filter out the columns that are different.
data3 = combined_data_1[different_cols]
 
# Merge the DataFrames
data = pd.merge(movie_budgets_df, data3, left_index=True,
                     right_index=True, how='inner')
data.shape

In [ ]:
data.head(3)

### 3.1 cleaning of the final dataset

#### 3.1.0 fixing structural issues
removal of the **$** from production_budget, domestic_gross and worldwide_gross


In [ ]:
final_data['production_budget'] = final_data['production_budget'].str.replace('[\$\,]','',regex=True)
final_data['domestic_gross'] = final_data['domestic_gross'].str.replace('[\$\,]','',regex=True)
final_data['worldwide_gross'] = final_data['worldwide_gross'].str.replace('[\$\,]','',regex=True)


#### 3.1.1 changing data types 
production_budget, domestic_gross and worldwide_gross

In [ ]:
final_data['production_budget'] = pd.to_numeric(final_data['production_budget']) 
final_data['domestic_gross'] = pd.to_numeric(final_data['domestic_gross'])
final_data['worldwide_gross'] = pd.to_numeric(final_data['worldwide_gross'])

final_data.dtypes

In [ ]:
#final_data.drop('genre_count', axis=1, inplace=True)

### data after cleaning

In [ ]:
final_data.shape

In [ ]:
final_data.info()

## 4.0 Exploratory Data Analysis

### 4.0.1 Profit Calculation

I now want to discuss the relationship between a movie's budget and how much it earns at the box office. I'll focus on the **Domestic gross**, which includes all ticket sales revenue from US and Canadian theaters.

I'll use the Pearson correlation coefficient to measure the linear correlation   between the production  budget and box office domestic gross.


This result indicates a correlation of **0.6** between production budget and the domestic gross.
This implies a **moderate** strength of relationship.

In [ ]:
final_data['domestic_profit'] = final_data['domestic_gross'] - final_data['production_budget']
correlation_df = final_data.iloc[:, [6,2,3,4,10]].head(13)
correlation = final_data.corr()
plt.figure(figsize=(15,5))
sns.heatmap(correlation, xticklabels = correlation.columns,
            yticklabels = correlation.columns,
            cmap = 'coolwarm', annot = True);
correlation

###  4.0.2 Genre Analysis

#### How are genres changing over time?

I've analyzed genre ties in this section. Because most of the movies in my datasets fit into more than one genre,it would be interesting to learn how the different types are related and perhaps check on how they affect the popularity of the movies compared against number votes.


In [ ]:
final_data.sort_values(by='release_date', ascending=True)

In [ ]:
genre_frequncy = dict(final_data['genres'].str.split(',', expand=True).stack().value_counts())

In [ ]:
fig, ax = plt.subplots(figsize =(10, 7))
explode = (0.0, 0.1, 0.2, 0.3, 0.2, 0.0,0.1,
           0.0, 0.2, 0.3, 0.0, 0.0,0.1, 0.0,
           0.2, 0.3, 0.5, 0.6,0.7, 0.8, 0.9, 1.0)

plt.pie(genre_frequncy.values(), labels=genre_frequncy.keys(),
        startangle=90, autopct='%.1f%%', colors=plt.cm.Set2.colors,
        explode = explode)
ax.set_title('Genre count')

plt.show();

Drama movies had the highest frequency counts and it was the most distributed genre across the combinations.

#### Which genres have the highest correlation between budget and market success?

To find the most expensive movies based on production budget, I sort my production budget in descending order and checked for the top 100.

In [ ]:
top_expensive_genre = final_data.sort_values(by=['production_budget'], ascending=False,).head(100)

In [ ]:
genres_in_top_expensive_movies = dict(top_expensive_genre['genres'].str.split(',', expand=True).stack().value_counts())

wordcloud = WordCloud(width = 1000, height = 500,
                     collocations = False, 
                      ).generate_from_frequencies(genres_in_top_expensive_movies)

plt.figure(figsize=(15,8))
plt.imshow(wordcloud);

This indicates, top perfoming genres are also the top expensive movies to produce.This means that in  majority of the cases, a larger budget has also resulted in a higher domestic box office gross.

Since most of the movies in the dataset have multiple Genres, I decided to split each of them to have a count plot and see how they are distributed.
From the value counts of the genres, drama had the highest value count while news had the least count.

### 4.0.3 Runtime Analysis.

In [ ]:
fig, ax = plt.subplots(figsize=(12, 8))
sns.countplot(data=data['runtime_minutes'], x=data['year'])
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, horizontalalignment='right')
plt.show()

From, this its seen that most movies averaged a runtime of 100 minutes.